## 라이브러리

In [1]:
import os
import json
from collections import defaultdict,Counter

## [Korquad](https://korquad.github.io/KorQuad%201.0/) evaluate.py의 후처리 함수 -> 사용하려면 밑의 @@@ 부분 주석 해제

In [2]:
import re
import string
def normalize_answer(s):    
    def remove_(text):
        ''' 불필요한 기호 제거 '''
        text = re.sub("'", " ", text)
        text = re.sub('"', " ", text)
        text = re.sub('《', " ", text)
        text = re.sub('》', " ", text)
        text = re.sub('<', " ", text)
        text = re.sub('>', " ", text) 
        text = re.sub('〈', " ", text)
        text = re.sub('〉', " ", text)   
        text = re.sub("\(", " ", text)
        text = re.sub("\)", " ", text)
        text = re.sub("‘", " ", text)
        text = re.sub("’", " ", text)      
        return text

    def white_space_fix(text):
        return ' '.join(text.split())

    def remove_punc(text):
        exclude = set(string.punctuation)
        return ''.join(ch for ch in text if ch not in exclude)

    def lower(text):
        return text.lower()

    return white_space_fix(remove_punc(lower(remove_(s))))

In [3]:
predict_bin_path = "input/level2-mrc-level2-nlp-09/outputs/SOTAandTestJsoncheck72600/predictions.json"
                                                                         #후처리 함수 적용시에 뼈대가 될 predictions.json -> 아무파일 가능
with open(predict_bin_path, "r",encoding='utf-8') as f:
    predict_bin = json.load(f)
    
key_list=list(predict_bin.keys())

In [4]:
file_path ='./ensemble6'                                                 #하드보팅 할 predictions.json들 있는 경로
json_file_path = []
json_files=[]
for json_path in os.listdir(file_path):
    if json_path.endswith('.json'):                                      #checkpoint 및 하위 폴더들 ignore (KLUE대회 태일님 앙상블 참고) 
        json_file_path.append(json_path)

for file in json_file_path:
    fp=os.path.join(file_path,file)
    with open(fp,"r",encoding='utf-8') as json_file:
        json_data=json.load(json_file)
        # for kl in key_list:                                   #@@@@@@@@후처리 함수 적용 후 하드보팅 하려면 주석해제 (위의 korquad 후처리 함수) @@@@@@
        #     json_data[kl]=normalize_answer(json_data[kl])
        json_files.append(json_data)

In [5]:
bin=defaultdict(list)                                                   #key값들이 있는 뼈대에 value들은 list로

for i in range(len(key_list)):                                          #defaultdict : key만 선언하면 value는 '' 빈 값
    bin[key_list[i]]
    
for json_file in json_files:                                            #하드보팅 할 값들을  list안에 전부 넣기
    for kl in key_list:
        bin[kl].append(json_file[kl])

counter=[]
for key in bin.keys():                                                  #counter 객체로 가장 빈도 수 높은 답변
    counter.append(Counter(bin[key]).most_common()[0][0])
    
for i in range(len(counter)):                                           #선언한 빈 객체에 빈도 수 높은 답변 채우기
    predict_bin[key_list[i]]=counter[i]

In [6]:
new_json_path = "ensemble.json"                                        # 하드보팅 앙상블 predictions.json 파일 저장

with open(new_json_path, 'w', encoding='utf-8') as file:
    json.dump(predict_bin, file)

# 추가
- most_common[0][0] -> 가장 많이 나오는 것으로 답변 저장 -> 동률의 경우 확률을 고려하지 못함 -> 추후에 nbest_predictions.json을 이용한 soft voting으로 해결
- [깃허브 프로젝트 참고](https://github.com/boostcampaitech3/level2-mrc-level2-nlp-09/issues/45)